# Load jdaviz app in mosviz configuration

In [ ]:
from jdaviz.app import Application

app = Application(configuration='mosviz')
app

In [ ]:
from astropy.utils.data import download_file

# This file is originally from https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/7495/stack/manga-7495-12704-LOGCUBE.fits.gz
# but has been modified to correct some inconsistencies in the way units are parsed

# Load cutout into the left image viewer
cutout_image_file = "/Users/javerbukh/Documents/data_for_mosviz/mosviz_nirspec_data_0.3/level3/mosviz_cutouts/227.fits"
app.load_data(cutout_image_file)

### Set image viewer (top left) to visualize cutout 227.fits

In [ ]:
fig_image = app.get_viewer("mos-image-viewer").figure

In [ ]:
import bqplot
from regions import RectanglePixelRegion, PixCoord
from ipywidgets import VBox
import numpy as np
from bqplot import *

def region_to_vertices(region):
    p = region.to_polygon()
    return p.vertices.x, p.vertices.y

# Extract slit data from files and visualize

In [ ]:
# Load 2d spectrum data on the right image viewer
spectrum_2d_file = "/Users/javerbukh/Documents/data_for_mosviz/mosviz_nirspec_data_0.3/level3/f170lp-g235m_mos_observation-6-c0e0_s00227_cal.fits"
app.load_data(spectrum_2d_file)

### Set 2D Spectrum viewer (top right) to visualize f170lp-g235m_mos_observation-6-c0e0_s00227_cal.fits

In [ ]:
fig_2d = app.get_viewer("mos-2d-spectrum-viewer").figure

In [ ]:
from astropy.io import fits
hdu = fits.open(spectrum_2d_file)
header = hdu[1].header

In [ ]:
# Code to create SkyRegions

from regions import  RectangleSkyRegion,RectanglePixelRegion
from astropy.coordinates import Angle, SkyCoord
from astropy import units as u 
from astropy.wcs import WCS
def jwst_header_to_skyregion(header):
    s_region = header['S_REGION']
    footprint = s_region.split("POLYGON ICRS")[1].split()
    ra = np.array(footprint[::2], dtype=np.float)
    dec = np.array(footprint[1::2], dtype=np.float)
        
    # Find center of slit
    cra = (max(ra) + min(ra))/2 
    cdec = (max(dec) + min(dec))/2
    
    # Find center as skycoord
    skycoord = SkyCoord(cra, cdec,
                        unit=(u.Unit(u.deg),
                              u.Unit(u.deg)))
    
    # Puts corners of slit into skycoord object
    corners = SkyCoord(ra, dec, unit="deg")
    
    # Compute length and width
    length = corners[0].separation(corners[1])
    width = corners[1].separation(corners[2])
    length = Angle(length, u.deg)
    width = Angle(width, u.deg)
    
    skyregion = RectangleSkyRegion(center=skycoord, width=width, height=length)
    return skyregion

In [ ]:
# Set WCS

sky_region = jwst_header_to_skyregion(header)
header_copy = header.copy()
header_copy['WCSAXES'] = 2
for key in header:
    if '3' in key:
        header_copy.remove(key)
wcs_2d = WCS(header_copy)


In [ ]:
# Use wcs of image viewer to scale slit dimensions correctly
image_hdu = fits.open(cutout_image_file)
wcs_image = WCS(image_hdu[0].header)

pixel_region = sky_region.to_pixel(wcs_image)


In [ ]:
# Create polygon region from the pixel region and set vertices
pix_rec = pixel_region.to_polygon()

x_coords = pix_rec.vertices.x
y_coords = pix_rec.vertices.y

# Create LinearScale that is the same size as the image viewer
scales={'x': fig_image.interaction.x_scale, 'y':fig_image.interaction.y_scale}

# Create slit
patch2 = bqplot.Lines(x=x_coords, y=y_coords, scales=scales, fill='none', colors = ["red"], stroke_width=2, close_path=True)

# Visualize slit on the figure
fig_image.marks = fig_image.marks + [patch2]
fig_image